In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

## Define dataset and variables

In [2]:
case = "CLM_default"
path = "/glade/scratch/jinmuluo/archive/CLM_default/lnd/hist/" 
start_date = date(2010, 1, 1)
end_date = date(2015, 12, 1)

# Because out model simulation results are in monthly average
delta_months = (end_date.year - start_date.year)*12 + end_date.month - start_date.month + 1

In [3]:
clmvar = ['NDEP_TO_SMINN', 'NFIX', 'FFIX_TO_SMINN', 'FERT_TO_SMINN', 'NITRATE_N_TO_SMINN', 'F_CANOPY_TO_SOIL',
          'F_N2O_NIT', 'F_NOx_NIT', 'F_N2O_DENIT', 'F_NOx_DENIT', 'F_N2_DENIT', 'SMIN_NO3_RUNOFF', 'SMIN_NO3_LEACHED', 
          'WOOD_HARVESTN', 'CROPPROD1N_LOSS', 'COL_FIRE_NLOSS',
          'F_NIT', 'F_DENIT', 'F_N2_DENIT', 'area', 'landfrac']

## Read NetCDF

In [4]:
CLM = []
for i in range(delta_months):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    CLM.append(path + case + ".clm2." + "h0" + "." + month +".nc")
    
def preprocess(ds, fields = clmvar):
    return(ds[fields])

def fix_time(ds):  
    date0 = ds['time'][0].values
    date1 = ds['time'][-1].values
    # ds['time'] =xr.cftime_range(str(yr0),periods=ndays,freq='D')
    ds['time'] = pd.date_range(str(date0),str(date1),freq='MS') 
    
    return ds

ds_MID = fix_time(xr.open_mfdataset(CLM, decode_times=True, preprocess=preprocess))

## Nitrogen Input

In [5]:
DEP = (ds_MID["NDEP_TO_SMINN"] * ds_MID['area']*ds_MID ['landfrac']*1e6).sum(dim=['lat', 'lon'])
NFIX = (ds_MID["NFIX"] * ds_MID['area']*ds_MID ['landfrac']*1e6).sum(dim=['lat', 'lon'])
FFIX = (ds_MID["FFIX_TO_SMINN"] * ds_MID['area']*ds_MID ['landfrac']*1e6).sum(dim=['lat', 'lon'])
FERT = (ds_MID["FERT_TO_SMINN"] * ds_MID['area']*ds_MID ['landfrac']*1e6).sum(dim=['lat', 'lon'])
FERT2 = (ds_MID["NITRATE_N_TO_SMINN"] * ds_MID['area']*ds_MID ['landfrac']*1e6).sum(dim=['lat', 'lon'])
CANOPY = (ds_MID["F_CANOPY_TO_SOIL"] * ds_MID['area']*ds_MID ['landfrac']*1e6).sum(dim=['lat', 'lon'])

dep = 0.0; nfix = 0.0; ffix=0.0; fert=0.0; fert2=0.0; canopy=0.0;

# unit transfer form gN/s to Tg/year
for i in range(len(ds_MID.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    seconds = (t2-t).days * 24 * 3600
    
    dep = dep + DEP[i].values * seconds* 1e-12
    nfix = nfix + NFIX[i].values * seconds* 1e-12
    ffix = ffix + FFIX[i].values * seconds* 1e-12
    fert = fert + FERT[i].values * seconds* 1e-12
    fert2 = fert2 + FERT2[i].values * seconds* 1e-12
    canopy = canopy + CANOPY[i].values * seconds* 1e-12
    
print("Depostion: ", dep, "TgN/year")
print("Nfixation: ", nfix + ffix, "TgN/year")
print("NH4 fertilizer: ", fert, "TgN/year")
print("NO3 fertilizer: ", fert2, "TgN/year")
print("CANOPY recycle: ", canopy, "TgN/year")

Depostion:  493.4951582472002 TgN/year
Nfixation:  702.8953198811998 TgN/year
NH4 fertilizer:  670.1770752499689 TgN/year
NO3 fertilizer:  0.0 TgN/year
CANOPY recycle:  0.0 TgN/year


## NOx emission

In [6]:
ds_nit = ds_MID["F_NOx_NIT"] * ds_MID['area']*ds_MID['landfrac']*1e6
ds_nit = ds_nit.sum(dim=['lat', 'lon'])

ds_denit = ds_MID["F_NOx_DENIT"] * ds_MID['area']*ds_MID['landfrac']*1e6
ds_denit = ds_denit.sum(dim=['lat', 'lon'])

NOx_nit = 0.0
NOx_denit = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(ds_MID.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    NOx_nit = NOx_nit + ds_nit[i].values * day * 24 * 3600
    NOx_denit = NOx_denit + ds_denit[i].values * day * 24 * 3600
    
NOx_nit = NOx_nit * 1e-12
NOx_denit = NOx_denit * 1e-12
print("NOx from nitrification: ", NOx_nit, "TgN/year")
print("NOx from denitrification: ", NOx_denit, "TgN/year")
print("Total NOx emission:", NOx_nit + NOx_denit, "TgN/year")

NOx from nitrification:  34.640347952024996 TgN/year
NOx from denitrification:  42.1995557565 TgN/year
Total NOx emission: 76.83990370852499 TgN/year


## N2O emission

In [7]:
ds_nit = ds_MID["F_N2O_NIT"] * ds_MID['area']*ds_MID['landfrac']*1e6
ds_nit = ds_nit.sum(dim=['lat', 'lon'])

ds_denit = ds_MID["F_N2O_DENIT"] * ds_MID['area']*ds_MID['landfrac']*1e6
ds_denit = ds_denit.sum(dim=['lat', 'lon'])

N2O_nit = 0.0
N2O_denit = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(ds_MID.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    N2O_nit = N2O_nit + ds_nit[i].values * day * 24 * 3600
    N2O_denit = N2O_denit + ds_denit[i].values * day * 24 * 3600
    
N2O_nit = N2O_nit * 1e-12
N2O_denit = N2O_denit * 1e-12
print("N2O from nitrification: ", N2O_nit, "TgN/year")
print("N2O from denitrification: ", N2O_denit, "TgN/year")
print("Total N2O emisison: ", N2O_nit + N2O_denit, "TgN/year")

N2O from nitrification:  31.573862755425 TgN/year
N2O from denitrification:  48.204947421675 TgN/year
Total N2O emisison:  79.7788101771 TgN/year


## N2 emission

In [8]:
ds_denit = ds_MID["F_N2_DENIT"] * ds_MID['area']*ds_MID['landfrac']*1e6
ds_denit = ds_denit.sum(dim=['lat', 'lon'])

N2_denit = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(ds_MID.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    N2_denit = N2_denit + ds_denit[i].values * day * 24 * 3600
    
N2_denit = N2_denit * 1e-12
print("N2 from denitrification: ", N2_denit, "TgN/year")

N2 from denitrification:  550.5602793767999 TgN/year


## Other loss terms

In [9]:
ds_runoff = (ds_MID['SMIN_NO3_RUNOFF'] + ds_MID['SMIN_NO3_LEACHED']) * ds_MID['area']*ds_MID['landfrac']*1e6
ds_wood_harvest = ds_MID['WOOD_HARVESTN'] * ds_MID['area']*ds_MID['landfrac']*1e6
ds_crop_harvest = ds_MID['CROPPROD1N_LOSS']* ds_MID['area']*ds_MID['landfrac']*1e6
ds_fire = ds_MID['COL_FIRE_NLOSS']* ds_MID['area']*ds_MID['landfrac']*1e6

ds_runoff  = ds_runoff .sum(dim=['lat', 'lon'])
ds_crop_harvest = ds_crop_harvest.sum(dim=['lat', 'lon'])
ds_wood_harvest = ds_wood_harvest.sum(dim=['lat', 'lon'])
ds_fire = ds_fire.sum(dim=['lat', 'lon'])

runoff = 0.0; crop_harvest = 0.0; wood_harvest=0.0; fire = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(ds_MID.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    runoff = runoff + ds_runoff[i].values * day * 24 * 3600
    crop_harvest = crop_harvest + ds_crop_harvest[i].values * day * 24 * 3600
    wood_harvest = wood_harvest + ds_wood_harvest[i].values * day * 24 * 3600
    fire = fire + ds_fire[i].values * day * 24 * 3600
    
runoff = runoff * 1e-12
crop_harvest = crop_harvest * 1e-12
wood_harvest = wood_harvest * 1e-12
fire = fire *1e-12

print("Runoff: ", runoff, "TgN/year")
print("Crop harvest: ", crop_harvest, "TgN/year")
print("Wood harvest: ", wood_harvest, "TgN/year")
print("Fire: ", fire, "TgN/year")

Runoff:  220.33111479299998 TgN/year
Crop harvest:  220.5113915466 TgN/year
Wood harvest:  13.9982354172 TgN/year
Fire:  280.87487314919997 TgN/year
